# Disclaimer & Copyright

Copyright 2024 Forusone : shins777@gmail.com

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at

https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

# Gecko Multimodal Embeddings - Korean text embedding test
* This notebook explains how to use Korean embeddings and understand vectorization.
* Refer to the link for more information about the embeddings.
 * https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/text-embeddings
 * https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/multimodal-embeddings

# Configuration
## Install python packages
* Vertex AI SDK for Python
  * https://cloud.google.com/python/docs/reference/aiplatform/latest

In [1]:
%pip install --upgrade --quiet google-cloud-aiplatform

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 10.6 MB/s eta 0:00:00


In [2]:
from IPython.display import display, Markdown

## Authentication to access to the GCP & Google drive

* Use OAuth to access the GCP environment.
 * Refer to the authentication methods in GCP : https://cloud.google.com/docs/authentication?hl=ko

In [3]:
#  For only colab to authenticate to get an access to the GCP.
import sys

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

* Mount to the google drive to access the .ipynb files in the repository.



In [4]:
# To access contents in Google drive

if "google.colab" in sys.modules:
  from google.colab import drive
  drive.mount('/content/drive')

Mounted at /content/drive


# Set the environment on GCP Project
* Configure project information
  * Model name : LLM model name : https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models
  * Project Id : prodect id in GCP
  * Region : region name in GCP

In [5]:
MODEL_NAME="gemini-1.5-flash"
PROJECT_ID="ai-hangsik"
REGION="asia-northeast3"

### Vertex AI initialization
Configure Vertex AI and access to the foundation model.
* Vertex AI initialization : aiplatform.init(..)
  * https://cloud.google.com/python/docs/reference/aiplatform/latest#initialization

In [6]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part
import vertexai.preview.generative_models as generative_models

# Initalizate the current vertex AI execution environment.
vertexai.init(project=PROJECT_ID, location=REGION)

# Access to the generative model.
#model = GenerativeModel(MODEL_NAME)

## Functions to evaluate similarity

In [11]:
import base64

def get_encoded_content(location_type, location, mime_type ):
  """
  Get the encoded content object.

  location_type :
    The type of the location. ( local or GCS )
  location :
    The file location of the content.
  mime_type :
    The mime type of the content.

  Returns:
    The encoded content object.

  """

  content_obj = None

  if location_type == "local":
    with open(location, 'rb') as f:
      raw_obj = base64.b64encode(f.read()).decode('utf-8')
      content_obj = Part.from_data(data=base64.b64decode(raw_obj), mime_type=mime_type)

  elif location_type == "GCS":
        content_obj = Part.from_uri(location, mime_type=mime_type)
  else:
    raise ValueError("Invalid location type.")

  return content_obj

## Multimodal embedding

* https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/multimodal-embeddings

In [13]:
import vertexai
from vertexai.vision_models import Image, MultiModalEmbeddingModel

def get_gecko_embeddings(model_name, image_path, contextual_text):

  model = MultiModalEmbeddingModel.from_pretrained(model_name)
  image = Image.load_from_file(image_path)

  embeddings = model.get_embeddings(
      image=image_path,
      contextual_text=contextual_text,
  )
  print(f"Image Embedding: {embeddings.image_embedding}")
  print(f"Text Embedding: {embeddings.text_embedding}")

  return embeddings


In [ ]:

model_name = "multimodalembedding"
contextual_text = "A cat is riding a motorcycle down the road."

repository_root = "/content/drive/MyDrive/projects/google_gen_ai_sample"
file_path = "/contents/images/cat_bike.png"
image_path = repository_root + file_path

embeddings = get_gecko_embeddings(model_name,image_path, contextual_text )
